<a href="https://colab.research.google.com/github/aditya02shah/MusicGeneration/blob/main/MusicGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
  !pip install mitdeeplearning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##Importing Libaries

In [ ]:
import tensorflow as tf
import mitdeeplearning as mdl
import numpy as np
import os
import time
from IPython import display as ipythondisplay
from tqdm import tqdm

##Dataset

In [ ]:
songs = mdl.lab1.load_training_data()

Found 817 songs in text


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
!apt-get -y install abcmidi timidity

In [ ]:
example_song = songs[0]
print("\nExample song: ")
print(example_song)


Example song: 
X:1
T:Alexander's
Z: id:dc-hornpipe-1
M:C|
L:1/8
K:D Major
(3ABc|dAFA DFAd|fdcd FAdf|gfge fefd|(3efe (3dcB A2 (3ABc|!
dAFA DFAd|fdcd FAdf|gfge fefd|(3efe dc d2:|!
AG|FAdA FAdA|GBdB GBdB|Acec Acec|dfaf gecA|!
FAdA FAdA|GBdB GBdB|Aceg fefd|(3efe dc d2:|!


In [ ]:
waveform = mdl.lab1.play_song(example_song)
if waveform:
   ipythondisplay.display(waveform)


In [ ]:
songs_joined = "\n\n".join(songs)

# Finding all unique characters in the joined string
vocab = sorted(set(songs_joined))
print("There are", len(vocab), "unique characters in the dataset")

There are 83 unique characters in the dataset


In [ ]:
vocab

##Data Preprocessing

####Vectorise the data

In [ ]:
char_to_index = dict((j,i) for i,j in enumerate(vocab))
print(char_to_index)

{'\n': 0, ' ': 1, '!': 2, '"': 3, '#': 4, "'": 5, '(': 6, ')': 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '<': 23, '=': 24, '>': 25, 'A': 26, 'B': 27, 'C': 28, 'D': 29, 'E': 30, 'F': 31, 'G': 32, 'H': 33, 'I': 34, 'J': 35, 'K': 36, 'L': 37, 'M': 38, 'N': 39, 'O': 40, 'P': 41, 'Q': 42, 'R': 43, 'S': 44, 'T': 45, 'U': 46, 'V': 47, 'W': 48, 'X': 49, 'Y': 50, 'Z': 51, '[': 52, ']': 53, '^': 54, '_': 55, 'a': 56, 'b': 57, 'c': 58, 'd': 59, 'e': 60, 'f': 61, 'g': 62, 'h': 63, 'i': 64, 'j': 65, 'k': 66, 'l': 67, 'm': 68, 'n': 69, 'o': 70, 'p': 71, 'q': 72, 'r': 73, 's': 74, 't': 75, 'u': 76, 'v': 77, 'w': 78, 'x': 79, 'y': 80, 'z': 81, '|': 82}


In [ ]:
index_to_char = dict((i,j) for i,j in enumerate(vocab))
print(index_to_char)

{0: '\n', 1: ' ', 2: '!', 3: '"', 4: '#', 5: "'", 6: '(', 7: ')', 8: ',', 9: '-', 10: '.', 11: '/', 12: '0', 13: '1', 14: '2', 15: '3', 16: '4', 17: '5', 18: '6', 19: '7', 20: '8', 21: '9', 22: ':', 23: '<', 24: '=', 25: '>', 26: 'A', 27: 'B', 28: 'C', 29: 'D', 30: 'E', 31: 'F', 32: 'G', 33: 'H', 34: 'I', 35: 'J', 36: 'K', 37: 'L', 38: 'M', 39: 'N', 40: 'O', 41: 'P', 42: 'Q', 43: 'R', 44: 'S', 45: 'T', 46: 'U', 47: 'V', 48: 'W', 49: 'X', 50: 'Y', 51: 'Z', 52: '[', 53: ']', 54: '^', 55: '_', 56: 'a', 57: 'b', 58: 'c', 59: 'd', 60: 'e', 61: 'f', 62: 'g', 63: 'h', 64: 'i', 65: 'j', 66: 'k', 67: 'l', 68: 'm', 69: 'n', 70: 'o', 71: 'p', 72: 'q', 73: 'r', 74: 's', 75: 't', 76: 'u', 77: 'v', 78: 'w', 79: 'x', 80: 'y', 81: 'z', 82: '|'}


In [ ]:
print('{')
for char,_ in zip(char_to_index, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char_to_index[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  '#' :   4,
  "'" :   5,
  '(' :   6,
  ')' :   7,
  ',' :   8,
  '-' :   9,
  '.' :  10,
  '/' :  11,
  '0' :  12,
  '1' :  13,
  '2' :  14,
  '3' :  15,
  '4' :  16,
  '5' :  17,
  '6' :  18,
  '7' :  19,
  ...
}


In [ ]:
def vectorize_string(string):
    vector = []
    for char in string:
#         print(type(char))
        vector .append(char_to_index[char])
    return np.array(vector)

vectorized_songs = vectorize_string(songs_joined)
# print(vectorized_songs.shape)

In [ ]:
print ('{} ---- characters mapped to int ----> {}'.format(repr(songs_joined[:10]), vectorized_songs[:10]))
# check that vectorized_songs is a numpy array
assert isinstance(vectorized_songs, np.ndarray), "returned result should be a numpy array"

'X:1\nT:Alex' ---- characters mapped to int ----> [49 22 13  0 45 22 26 67 60 79]


####Training Examples and Targets

In [ ]:
def get_batch(vectorized_songs, seq_length, batch_size):
    # the length of the vectorized songs string
    n = vectorized_songs.shape[0] - 1
    # randomly choose the starting indices for the examples in the training batch
    idx = np.random.choice(n-seq_length, batch_size)
    input_batch = [vectorized_songs[i : i+seq_length] for i in idx]
    output_batch = [vectorized_songs[i+1 : i+seq_length+1] for i in idx]

    # x_batch, y_batch provide the true inputs and targets for network training
    x_batch = np.reshape(input_batch, [batch_size, seq_length])
    y_batch = np.reshape(output_batch, [batch_size, seq_length])
    return x_batch, y_batch
test_args = (vectorized_songs, 10, 2)
if not mdl.lab1.test_batch_func_types(get_batch, test_args) or \
   not mdl.lab1.test_batch_func_shapes(get_batch, test_args) or \
   not mdl.lab1.test_batch_func_next_step(get_batch, test_args):
    print("Could not pass tests")
else:
    print("Passed all tests!")

[PASS] test_batch_func_types
[PASS] test_batch_func_shapes
[PASS] test_batch_func_next_step
[PASS] passed all tests!


In [ ]:
x_batch, y_batch = get_batch(vectorized_songs, seq_length=5, batch_size=1)

for i, (input_idx, target_idx) in enumerate(zip(np.squeeze(x_batch), np.squeeze(y_batch))):
    print("Step {:3d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(index_to_char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(index_to_char[target_idx])))

Step   0
  input: 14 ('2')
  expected output: 58 ('c')
Step   1
  input: 58 ('c')
  expected output: 1 (' ')
Step   2
  input: 1 (' ')
  expected output: 27 ('B')
Step   3
  input: 27 ('B')
  expected output: 14 ('2')
Step   4
  input: 14 ('2')
  expected output: 59 ('d')


##LSTM Model

In [ ]:
def LSTM(rnn_units):
    return tf.keras.layers.LSTM(rnn_units, return_sequences=True, recurrent_initializer='glorot_uniform',
                                recurrent_activation='sigmoid',stateful=True,)

In [ ]:
def build_model_1(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
    # Layer 1: Embedding layer to transform indices into dense vectors
    tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),

    # Layer 2: LSTM with `rnn_units` number of units.
    LSTM(rnn_units),
    # Layer 3: Dense (fully-connected) layer that transforms the LSTM output
    tf.keras.layers.Dense(vocab_size)
    ])
    return model

model = build_model_1(len(vocab), embedding_dim=256, rnn_units=1024, batch_size=32)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (32, None, 256)           21248     
                                                                 
 lstm (LSTM)                 (32, None, 1024)          5246976   
                                                                 
 dense (Dense)               (32, None, 83)            85075     
                                                                 
Total params: 5,353,299
Trainable params: 5,353,299
Non-trainable params: 0
_________________________________________________________________


In [ ]:
x, y = get_batch(vectorized_songs, seq_length=100, batch_size=32)
pred = model(x)
print("Input shape:      ", x.shape, " # (batch_size, sequence_length)")
print("Prediction shape: ", pred.shape, "# (batch_size, sequence_length, vocab_size)")

Input shape:       (32, 100)  # (batch_size, sequence_length)
Prediction shape:  (32, 100, 83) # (batch_size, sequence_length, vocab_size)


##Training the model

In [ ]:
def compute_loss(labels, logits):
    loss = tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
    return loss

example_batch_loss = compute_loss(y, pred)

print("Prediction shape: ", pred.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())


Prediction shape:  (32, 100, 83)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.419444


####Defining Hyperparameteres

In [ ]:
#Optimization parameters
num_training_iterations = 10000
batch_size = 4
seq_length = 100
learning_rate = 5e-3

# Model parameters:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

# Checkpoint location:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "my_ckpt")

In [ ]:
model = build_model_1(vocab_size, embedding_dim, rnn_units, batch_size)
optimizer = tf.keras.optimizers.Adam()
def train_step(x,y):
    with tf.GradientTape() as tape:
        y_pred = model(x)
        loss = compute_loss(y, y_pred)
    grad = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grad, model.trainable_variables))
    return loss.numpy().mean()
losses = []
for i in tqdm(range(0,num_training_iterations)):
    x, y = get_batch(vectorized_songs, seq_length, batch_size)
    loss = train_step(x,y)
    losses.append(loss)

    if i % 100 == 0:
        model.save_weights(checkpoint_prefix)

model.save_weights(checkpoint_prefix)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(losses)

In [ ]:
model = build_model_1(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

model.summary()

In [ ]:
def generate_text(model, start_string, generation_length=1000):
    input_eval = [char_to_index[i] for i in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []
    model.reset_states()
    tqdm._instances.clear()

    for i in tqdm(range(generation_length)):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
#         print(predicted_id)
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(index_to_char[predicted_id])
    return (start_string + ''.join(text_generated))

In [ ]:
generated_text = generate_text(model, start_string="C", generation_length=10000)

In [ ]:
print(generated_text[:1000])

In [ ]:
generated_songs = mdl.lab1.extract_song_snippet(generated_text)
m = 0;
for i, song in enumerate(generated_songs):
  # Synthesize the waveform from a song
    waveform = mdl.lab1.play_song(song)
    m+=1
    if waveform:
        print("Generated song", i)
        ipythondisplay.display(waveform)
        if m>3:
            break

##Saving and Loading Model

In [ ]:
model.save('LSTM_model.h5')

In [ ]:
music_model=tf.keras.models.load_model('/content/LSTM_model.h5')

In [ ]:
music_model.summary()

In [ ]:
demo_text = generate_text(model, start_string="E", generation_length=1000)

In [ ]:
generated_songs = mdl.lab1.extract_song_snippet(demo_text)
m = 0;
for i, song in enumerate(generated_songs):
    waveform = mdl.lab1.play_song(song)
    m+=1
    if waveform:
        print("Generated song", i)
        ipythondisplay.display(waveform)
        if m>3:
            break

##Converting to TFlite Model

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(music_model)
tflite_model = converter.convert()

In [ ]:
with tf.io.gfile.GFile('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

/usr/local/lib/python3.10/dist-packages/pexpect/popen_spawn.py:60: DeprecationWarning: setDaemon() is deprecated, set the daemon attribute instead
  self._read_thread.setDaemon(True)


Mounted at /content/gdrive/


In [ ]:
model1=tf.keras.models.load_model('/content/gdrive/MyDrive/PBL/LSTM_model.h5')

In [ ]:
demo_text1 = generate_text(model1, start_string="Z", generation_length=1000)

100%|██████████| 1000/1000 [00:13<00:00, 74.68it/s]


In [ ]:
generated_songs = mdl.lab1.extract_song_snippet(demo_text1)
m = 0;
for i, song in enumerate(generated_songs):
  # Synthesize the waveform from a song
    waveform = mdl.lab1.play_song(song)
    m+=1
  # If its a valid song (correct syntax), lets play it!
    if waveform:
        print("Generated song", i)
        ipythondisplay.display(waveform)
        if m>3:
            break